In [1]:
import numpy as np
import tensorflow as tf
import time

In [2]:
random_fix = True

In [3]:
class Dataset():
    def __init__(self, mode):
        self.mode = mode
        self.input_dim = self.output_dim = self.train_count = 0
        self.train_xs = self.test_xs = self.validate_xs = []
        self.train_ys = self.test_ys = self.validate_ys = []
        self.target_names = []
    
    def get_train_data(self, batch_size, nth):
        if nth == 0:
            self.indices = np.arange(self.train_count)
            np.random.shuffle(self.indices)
        
        from_idx = nth * batch_size
        to_idx = (nth + 1) * batch_size
    
        train_X = self.train_xs[self.indices[from_idx:to_idx]]
        train_Y = self.train_ys[self.indices[from_idx:to_idx]]
        
        return train_X, train_Y    
    
    def get_test_data(self, validate=False, count=0):
        if validate:
            xs, ys = self.validate_xs, self.validate_ys
        else:
            xs, ys = self.test_xs, self.test_ys
        
        if count == 0:
            return xs, ys
        
        if count > len(xs): count = len(xs)
        
        indices = np.arange(len(xs))
        np.random.shuffle(indices)
        
        return xs[indices[0:count]], ys[indices[0:count]]
    
    def get_target_name(self, idxs):
        return self.target_names[idxs]
    
    def demonstrate(self, x, y, estimate, answer, probs):
        pass

In [4]:
class Model(object):
    def __init__(self, name, dataset):
        self.name = name
        self.dataset = dataset
    
    def train(self, epoch=10, batch_size=10):
        pass
    
    def test(self):
        pass
    
    def demonstrate(self, num):
        pass

In [5]:
from sklearn import datasets

class SudokuDataset(Dataset):
    pass

def unpack_map(line):
    quiz = np.zeros([81], dtype=np.int8)
    solution = np.zeros([81], dtype=np.int8)
    for n in range(81):
        quiz[n] = int(line[n])
        solution[n] = int(line[n+82])
    return quiz, solution

def fill_hints(quiz, solution, hint_cnt):
    for n in range(hint_cnt):
        for k in range(1000):
            pos = np.random.randint(81)
            if quiz[pos] == 0:
                quiz[pos] = solution[pos]
                break
    return quiz
    
def sudoku_init(self):
    Dataset.__init__(self, "sudoku")
    
    self.input_dim = 81
    self.output_dim = 810
    
    quizzes, solutions = [], []

    unit = 1000 # 1000
    
    for line in open("./data/sudoku.csv"):
        if line[0] == 'q': continue
        quiz, solution = unpack_map(line)
        hint_cnt = int((81-np.count_nonzero(quiz))/10)
        for n in range(10):
            quizzes.append(quiz)
            solutions.append(solution)
            quiz = fill_hints(quiz, solution, hint_cnt)
        if len(quizzes) >= 125 * unit: break

    xs = np.asarray(quizzes)
    ys = np.eye(10)[solutions].reshape(-1, 810)

    self.train_count = 100 * unit

    indices = np.arange(125 * unit)
    np.random.shuffle(indices)

    self.train_xs = xs[indices[0:100 * unit]]
    self.train_ys = ys[indices[0:100 * unit]]
    self.test_xs = xs[indices[100 * unit:120 * unit]]
    self.test_ys = ys[indices[100 * unit:120 * unit]]
    self.validate_xs = xs[indices[120 * unit:125 * unit]]
    self.validate_ys = ys[indices[120 * unit:125 * unit]]

def sudoku_demonstrate(self, x, y, estimate, answer, probs):
    print("sudoku demonstrate dummy")
    
SudokuDataset.__init__ = sudoku_init
SudokuDataset.demonstrate = sudoku_demonstrate

In [6]:
sd = SudokuDataset()

In [7]:
class MultiLayerPerceptronModel(Model):
    def build_loss_accuracy(self):
        pass

def mlp_init(self, name, dataset, hidden_dims, learning_rate=0.001):
    Model.__init__(self, name, dataset)
    build_graph(self, hidden_dims, learning_rate)

def init_rand_normal(in_dim, out_dim, rand_std=0.0300):
    if not random_fix:
        init = tf.random_normal([in_dim, out_dim], stddev=rand_std)
    else:
        init_64 = np.random.normal(0, rand_std, [in_dim, out_dim])
        init = init_64.astype('float32')

    return init

def build_graph(self, hidden_dims, learning_rate):
    with tf.variable_scope(self.name):
        build_resources(self, hidden_dims)
        build_neuralnet(self, hidden_dims)
        self.build_loss_accuracy()
        build_optimizer(self, learning_rate)
    built_saver(self)

def build_resources(self, hidden_dims):
    self.x = tf.placeholder("float", [None, self.dataset.input_dim])
    self.y = tf.placeholder("float", [None, self.dataset.output_dim])

    if random_fix: np.random.seed(9876)

    self.w_hids, self.b_hids = [], []
    
    prev_dim = self.dataset.input_dim

    for n in range(len(hidden_dims)):
        next_dim = hidden_dims[n]
        w = tf.Variable(init_rand_normal(prev_dim, next_dim))
        b = tf.Variable(tf.zeros([next_dim]))
        self.w_hids.append(w)
        self.b_hids.append(b)
        prev_dim = next_dim

    self.w_out = tf.Variable(init_rand_normal(prev_dim, self.dataset.output_dim))
    self.b_out = tf.Variable(tf.zeros([self.dataset.output_dim]))

def build_neuralnet(self, hidden_dims):
    hidden = self.x
    for n in range(len(hidden_dims)):
        hidden = tf.nn.relu(tf.matmul(hidden, self.w_hids[n]) + self.b_hids[n])
    self.output = tf.matmul(hidden, self.w_out) + self.b_out

def build_loss_accuracy(self):
    if self.dataset.mode == "regression":
        self.estimate = self.output[:,0]
        self.answer = self.y[:,0]
        diff = self.estimate - self.answer
        self.loss = tf.reduce_mean(tf.pow(diff, 2))
        self.accuracy = 1 - tf.reduce_mean(tf.abs(diff) / self.answer)
    elif self.dataset.mode == "binary":
        cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=self.y, logits=self.output)
        self.loss = tf.reduce_mean(cross_entropy)
        #probs = tf.nn.sigmoid(output)
        #estimate = tf.greater(probs, 0.5)
        self.estimate = tf.greater(self.output, 0)
        self.answer = tf.equal(self.y, 1.0)
        correct = tf.cast(tf.equal(self.estimate, self.answer), "float")
        self.accuracy = tf.reduce_mean(correct)
    elif self.dataset.mode == "select":
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=self.y, logits=self.output)
        self.loss = tf.reduce_mean(cross_entropy)
        #probs = tf.nn.softmax(output)
        #estimate = tf.argmax(probs, 1)
        self.estimate = tf.argmax(self.output, 1)
        self.answer = tf.argmax(self.y, 1)
        correct = tf.cast(tf.equal(self.estimate, self.answer), "float")
        self.accuracy = tf.reduce_mean(correct)

def build_optimizer(self, learning_rate):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    self.train_op = optimizer.minimize(self.loss)

def built_saver(self):
    var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
    self.saver = tf.train.Saver(var_list=var_list)

MultiLayerPerceptronModel.__init__ = mlp_init
MultiLayerPerceptronModel.build_loss_accuracy = build_loss_accuracy

In [8]:
def model_train(self, epoch_count=10, batch_size=10):
    if batch_size == 0:
        batch_size = self.dataset.train_count
        
    batch_count = int(self.dataset.train_count / batch_size)
    report_period = epoch_count / 10
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    if random_fix: np.random.seed(1945)
    
    time1 = time2 = int(time.time())
    
    print("Model {} train report:".format(self.name))
    
    for epoch in range(epoch_count):
        costs = []
        accs = []
        for n in range(batch_count):
            train_X, train_Y = self.dataset.get_train_data(batch_size, n)
            _, cost, acc = sess.run([self.train_op, self.loss, self.accuracy], \
                                    feed_dict={self.x:train_X, self.y:train_Y})
            costs.append(cost)
            accs.append(acc)
            
        if (epoch+1) % report_period == 0:
            validate_X, validate_Y = self.dataset.get_test_data(True, 30)
            acc = sess.run(self.accuracy, feed_dict={self.x:validate_X, self.y:validate_Y})
            time3 = int(time.time())
            print("    Epoch {}: cost={:5.3f}, accuracy={:5.3f}/{:5.3f} ({}/{} secs)". \
                  format(epoch+1, np.mean(costs), np.mean(accs), acc, time3-time2, time3-time1))
            time2 = time3

    print("")
    
    path = "params/{}.ckpt".format(self.name)
    self.saver.save(sess, path)
    sess.close()

def model_test(self):
    test_X, test_Y = self.dataset.get_test_data()

    sess = tf.Session()
    path = "params/{}.ckpt".format(self.name)
    self.saver.restore(sess, path)
    
    time1 = int(time.time())
    acc = sess.run(self.accuracy, feed_dict={self.x:test_X, self.y:test_Y})
    time2 = int(time.time())
    
    print("Model {} test report: accuracy = {:5.3f}, ({} secs)".format(self.name, acc, time2-time1))
    print("")
    
    sess.close()

def model_demonstrate(self, num=10, batch=False):
    demo_X, demo_Y = self.dataset.get_test_data(False, num)

    sess = tf.Session()
    path = "params/{}.ckpt".format(self.name)
    self.saver.restore(sess, path)
    
    print("Model {} Demonstration".format(self.name))
    estimate, answer, probs = sess.run([self.estimate, self.answer, self.probs], feed_dict={self.x:demo_X, self.y:demo_Y})
    if batch:
        self.dataset.demonstrate(demo_X, estimate, answer, probs)
    else:
        for n in range(len(demo_X)):
            self.dataset.demonstrate(demo_X[n], demo_Y[n], estimate[n], answer[n], probs[n])
        print("")
    
    sess.close()
    
Model.train = model_train
Model.test = model_test
Model.demonstrate = model_demonstrate

In [74]:
class SudokuMlpModel(MultiLayerPerceptronModel):
    pass

def sudoku_init(self, name, dataset, hidden_dims, learning_rate=0.001):
    MultiLayerPerceptronModel.__init__(self, name, dataset, hidden_dims, learning_rate)

def sudoku_build_loss_accuracy(self):
    self.temp_labels = tf.reshape(self.y, [-1, 81, 10])
    self.temp_logits = tf.reshape(self.output, [-1, 81, 10])
    self.temp_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=self.temp_labels, logits=self.temp_logits)
    self.loss = tf.reduce_mean(self.temp_cross_entropy)
    self.probs = tf.nn.softmax(self.temp_logits)
    self.test = tf.argmax(self.temp_logits, 2)
    self.estimate = tf.argmax(self.temp_logits, 2)
    self.answer = tf.argmax(self.temp_labels, 2)
    self.temp_correct = tf.cast(tf.equal(self.estimate, self.answer), "float")
    self.accuracy = tf.reduce_mean(self.temp_correct)

def sudoku_train(self, epoch_count=2, batch_size=10):
    if batch_size == 0:
        batch_size = self.dataset.train_count
        
    batch_count = int(self.dataset.train_count / batch_size)
    report_period = epoch_count / 10
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    if random_fix: np.random.seed(1945)
    
    time1 = time2 = int(time.time())
    
    print("Model {} train report:".format(self.name))
    
    for epoch in range(epoch_count):
        costs = []
        accs = []
        for n in range(batch_count):
            train_X, train_Y = self.dataset.get_train_data(batch_size, n)
            temp_test, _, x, y, labels, output, logits, entropy, cost, probs, estimate, answer, correct, acc = \
                sess.run([self.test, self.train_op, self.x, self.y, self.temp_labels, self.output, self.temp_logits, \
                          self.temp_cross_entropy, self.loss, self.probs, self.estimate, \
                          self.answer, self.temp_correct, self.accuracy], \
                                    feed_dict={self.x:train_X, self.y:train_Y})
            #_, cost, acc = sess.run([self.train_op, self.loss, self.accuracy], \
            #                        feed_dict={self.x:train_X, self.y:train_Y})
            costs.append(cost)
            accs.append(acc)
        
        if (epoch+1) % report_period == 0:
            validate_X, validate_Y = self.dataset.get_test_data(True, 30)
            acc = sess.run(self.accuracy, feed_dict={self.x:validate_X, self.y:validate_Y})
            time3 = int(time.time())
            print("    Epoch {}: cost={:5.3f}, accuracy={:5.3f}/{:5.3f} ({}/{} secs)". \
                  format(epoch+1, np.mean(costs), np.mean(accs), acc, time3-time2, time3-time1))
            time2 = time3

    print("train_X", np.shape(train_X))
    print("x", np.shape(x))
    print(x[0])
    print("y", np.shape(y))
    #print(y[0])
    print("labels", np.shape(labels))
    #print(labels[0])
    print("output", np.shape(output))
    #print(output[0])
    print("logits", np.shape(logits))
    #print(logits[0])
    print("probs", np.shape(probs))
    #print(probs[0])
    print("entropy", np.shape(entropy))
    print(entropy[0])
    print("temp_test", np.shape(temp_test))
    print(temp_test[0])
    print("estimate", np.shape(estimate))
    print(estimate[0])
    print("answer", np.shape(answer))
    print(answer[0])
    print("correct", np.shape(correct))
    print(correct[0])
    print("acc", np.shape(acc))
    print(acc)

    print("")
    
    path = "params/{}.ckpt".format(self.name)
    self.saver.save(sess, path)
    sess.close()

SudokuMlpModel.__init__ = sudoku_init
SudokuMlpModel.train = sudoku_train
SudokuMlpModel.build_loss_accuracy = sudoku_build_loss_accuracy

In [76]:
m1 = SudokuMlpModel("sudoku-hidden-1", sd, [512])
m1.train(epoch_count=100, batch_size=10)

#m2 = SudokuMlpModel("sudoku-hidden-2", sd, [256, 128])
#m2.train(epoch_count=100, batch_size=100)

Model sudoku-hidden-1 train report:
    Epoch 10: cost=1.982, accuracy=0.243/0.249 (331/331 secs)
    Epoch 20: cost=1.755, accuracy=0.330/0.339 (331/662 secs)
    Epoch 30: cost=1.628, accuracy=0.406/0.400 (330/992 secs)
    Epoch 40: cost=1.571, accuracy=0.444/0.442 (331/1323 secs)
    Epoch 50: cost=1.546, accuracy=0.462/0.445 (328/1651 secs)
    Epoch 60: cost=1.533, accuracy=0.471/0.487 (331/1982 secs)
    Epoch 70: cost=1.526, accuracy=0.477/0.484 (330/2312 secs)
    Epoch 80: cost=1.521, accuracy=0.481/0.483 (327/2639 secs)
    Epoch 90: cost=1.518, accuracy=0.484/0.479 (326/2965 secs)
    Epoch 100: cost=1.516, accuracy=0.486/0.472 (327/3292 secs)
train_X (10, 81)
x (10, 81)
[ 0.  3.  8.  7.  5.  1.  4.  6.  2.  2.  1.  4.  9.  8.  0.  5.  3.  7.
  7.  6.  5.  4.  3.  2.  9.  8.  1.  3.  5.  6.  2.  7.  4.  1.  9.  8.
  1.  8.  9.  5.  6.  3.  2.  7.  4.  4.  7.  2.  1.  9.  8.  3.  5.  0.
  6.  4.  7.  3.  2.  0.  8.  1.  9.  5.  9.  1.  8.  4.  0.  6.  2.  3.
  0.  0.  3.  6.

In [83]:
Model.test = model_test
SudokuMlpModel.test = model_test
m1.test = model_test
print(m1)
m1.test()
#m2.test()

TypeError: model_test() missing 1 required positional argument: 'self'

In [93]:
def sudoku_demonstrate(self, x, y, estimate, answer, probs):
    unknown_cnt = 0
    known_mismatch_cnt = 0
    
    for n in range(81):
        if x[n] != 0:
            est = np.argmax(probs[n])
            if est != x[n]: known_mismatch_cnt += 1
            probs[n,:] = 0
        else: unknown_cnt += 1
            
    print('unknown_cnt', unknown_cnt)
    print('known_mismatch_cnt', known_mismatch_cnt)
    idx = np.argmax(probs)
    print("shape(probs)", np.shape(probs))
    print("idx(argmax)", idx)
    """
    pos = int(idx / 10)
    val = idx % 10
    ans = np.argmax(y[pos*10:(pos+1)*10])
    #print(idx, pos, val)
    print(x[pos], val, ans, probs[pos][val])
    print(probs[pos])
    """
    
SudokuDataset.__init__ = sudoku_init
SudokuDataset.demonstrate = sudoku_demonstrate

In [94]:
m1.demonstrate(10)
#m2.demonstrate(10)

INFO:tensorflow:Restoring parameters from params/sudoku-hidden-1.ckpt
Model sudoku-hidden-1 Demonstration
unknown_cnt 8
known_mismatch_cnt 27
shape(probs) (81, 10)
idx(argmax) 191
unknown_cnt 8
known_mismatch_cnt 31
shape(probs) (81, 10)
idx(argmax) 631
unknown_cnt 7
known_mismatch_cnt 34
shape(probs) (81, 10)
idx(argmax) 721
unknown_cnt 7
known_mismatch_cnt 31
shape(probs) (81, 10)
idx(argmax) 31
unknown_cnt 9
known_mismatch_cnt 40
shape(probs) (81, 10)
idx(argmax) 11
unknown_cnt 8
known_mismatch_cnt 42
shape(probs) (81, 10)
idx(argmax) 181
unknown_cnt 5
known_mismatch_cnt 34
shape(probs) (81, 10)
idx(argmax) 221
unknown_cnt 6
known_mismatch_cnt 31
shape(probs) (81, 10)
idx(argmax) 231
unknown_cnt 8
known_mismatch_cnt 34
shape(probs) (81, 10)
idx(argmax) 71
unknown_cnt 8
known_mismatch_cnt 33
shape(probs) (81, 10)
idx(argmax) 741

